<a href="https://colab.research.google.com/github/edx04/data-science/blob/main/WranglingArellano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
w_data = pd.read_csv('https://raw.githubusercontent.com/edx04/data-science/main/data/w_data.csv', sep=';')
p_data = pd.read_csv('https://raw.githubusercontent.com/edx04/data-science/main/data/p_data.csv', sep=';')

#1 Cuantos ID tienen en común. Nombrar cuales no tienen en común

In [3]:
result = pd.merge(w_data, p_data, on='ID')
result.head()

,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,ID,Target,Age,Sex,Race,Country,Education,Education-Num,Marital Status,Relationship
0,State-gov,77516.0,2174.0,0.0,40.0,49084,<=50K,39.0,Male,White,United-States,Bachelors,13.0,Never-married,Not-in-family
1,Self-emp-not-inc,83311.0,0.0,0.0,13.0,68553,<=50K,50.0,Male,White,United-States,Bachelors,13.0,Married-civ-spouse,Husband
2,Private,215646.0,0.0,0.0,40.0,143654,<=50K,38.0,Male,White,United-States,HS-grad,9.0,Divorced,Not-in-family
3,Private,234721.0,0.0,0.0,40.0,41510,<=50K,53.0,Male,Black,United-States,11th,7.0,Married-civ-spouse,Husband
4,Private,338409.0,0.0,0.0,40.0,195025,<=50K,28.0,Female,Black,Cuba,Bachelors,13.0,Married-civ-spouse,Wife


In [4]:
print(f"tienen en comun {result.ID.count()}")

tienen en comun 48842


In [5]:
diferente_id = pd.merge(w_data, p_data, on='ID',how='outer',indicator=True).query('_merge != "both"')
diferente_id

,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,ID,Target,Age,Sex,Race,Country,Education,Education-Num,Marital Status,Relationship,_merge
48842,Self-emp-inc,182148.0,0.0,0.0,60.0,1344995623,>50K.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [6]:
print(f"NO tienen en comun el id  {diferente_id.ID.values[0]}")

NO tienen en comun el id  1344995623


#2 Corregir los target a solo dos valores distintos ‘<=50k’ y ‘>50k’

In [7]:
result.Target.unique()

array([' <=50K', ' >50K', nan, ' <=50K.', ' >50K.'], dtype=object)

In [8]:
result[result['Target'].isnull()]

,Workclass,fnlwgt,Capital Gain,Capital Loss,Hours per week,ID,Target,Age,Sex,Race,Country,Education,Education-Num,Marital Status,Relationship
32561,NaN,NaN,NaN,NaN,NaN,37078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
result2 = result.drop(index=32561)

In [10]:
result2.Target.unique()

array([' <=50K', ' >50K', ' <=50K.', ' >50K.'], dtype=object)

In [11]:
result2.Target= result2.Target.apply(lambda x: '>50k' if x[1] == '>' else '<=50k')

In [12]:
result2.Target.unique()

array(['<=50k', '>50k'], dtype=object)

In [13]:
result2.Target.value_counts()

<=50k    37155
>50k     11686
Name: Target, dtype: int64

#3 Calcular la proporción que hay entre personas con salarios >50k respecto del total para Entre personas de distinto sexo, Raza y Educacion. Por ejemplo cuál es el porcentaje de mujeres con un salario mayor a 50K y compararlo con el de los hombres. Lo mismo para personas de distintas razas y Educación


In [14]:
values = result2.groupby(['Sex','Target']).count()['Workclass']
values

Sex      Target
 Female  <=50k     14423
         >50k       1769
 Male    <=50k     22732
         >50k       9917
Name: Workclass, dtype: int64

In [15]:
total = result2.groupby('Target').count()['Workclass']['>50k']
total

11686

In [16]:
#porcentaje de mujeres con un salario mayor a 50K
values[1]/total

0.15137771692623653

In [17]:
#porcentaje de hombres con un salario mayor a 50K
values[3]/total

0.8486222830737635

Raza


In [18]:
values_race = result2.groupby(['Race','Target']).count()['Workclass']
values_race

Race                 Target
 Amer-Indian-Eskimo  <=50k       415
                     >50k         55
 Asian-Pac-Islander  <=50k      1110
                     >50k        409
 Black               <=50k      4119
                     >50k        566
 Other               <=50k       356
                     >50k         50
 White               <=50k     31155
                     >50k      10606
Name: Workclass, dtype: int64

In [19]:
values_race.index

MultiIndex([(' Amer-Indian-Eskimo', '<=50k'),
            (' Amer-Indian-Eskimo',  '>50k'),
            (' Asian-Pac-Islander', '<=50k'),
            (' Asian-Pac-Islander',  '>50k'),
            (             ' Black', '<=50k'),
            (             ' Black',  '>50k'),
            (             ' Other', '<=50k'),
            (             ' Other',  '>50k'),
            (             ' White', '<=50k'),
            (             ' White',  '>50k')],
           names=['Race', 'Target'])

In [33]:
for index in values_race.index:
  if index[1] == '<=50k':
    continue
  print(f'proporcion de {index[0]} con un salario mayor a 50k: {values_race[index]/total}')


proporcion de  Amer-Indian-Eskimo con un salario mayor a 50k: 0.004706486393975698
proporcion de  Asian-Pac-Islander con un salario mayor a 50k: 0.034999144275201094
proporcion de  Black con un salario mayor a 50k: 0.04843402361800445
proporcion de  Other con un salario mayor a 50k: 0.004278623994523361
proporcion de  White con un salario mayor a 50k: 0.9075817217182954


Educacion

In [23]:
values_educacion = result2.groupby(['Education','Target']).count()['Workclass']
values_educacion

Education      Target
 10th          <=50k      1302
               >50k         87
 11th          <=50k      1720
               >50k         92
 12th          <=50k       609
               >50k         48
 1st-4th       <=50k       239
               >50k          8
 5th-6th       <=50k       482
               >50k         27
 7th-8th       <=50k       893
               >50k         62
 9th           <=50k       715
               >50k         41
 Assoc-acdm    <=50k      1188
               >50k        413
 Assoc-voc     <=50k      1539
               >50k        522
 Bachelors     <=50k      4712
               >50k       3312
 Doctorate     <=50k       163
               >50k        431
 HS-grad       <=50k     13281
               >50k       2503
 Masters       <=50k      1198
               >50k       1459
 Preschool     <=50k        82
               >50k          1
 Prof-school   <=50k       217
               >50k        617
 Some-college  <=50k      8815
               >5

In [35]:
for index in values_educacion.index:
  if index[1] == '<=50k':
    continue
  porcentaje = (values_educacion[index]/total)
  print(f'proporcion de {index[0]} con un salario mayor a 50k: {porcentaje} ')

proporcion de  10th con un salario mayor a 50k: 0.007444805750470649 
proporcion de  11th con un salario mayor a 50k: 0.007872668149922985 
proporcion de  12th con un salario mayor a 50k: 0.004107479034742427 
proporcion de  1st-4th con un salario mayor a 50k: 0.0006845798391237378 
proporcion de  5th-6th con un salario mayor a 50k: 0.002310456957042615 
proporcion de  7th-8th con un salario mayor a 50k: 0.005305493753208968 
proporcion de  9th con un salario mayor a 50k: 0.0035084716755091564 
proporcion de  Assoc-acdm con un salario mayor a 50k: 0.03534143419476297 
proporcion de  Assoc-voc con un salario mayor a 50k: 0.04466883450282389 
proporcion de  Bachelors con un salario mayor a 50k: 0.2834160533972275 
proporcion de  Doctorate con un salario mayor a 50k: 0.036881738832791375 
proporcion de  HS-grad con un salario mayor a 50k: 0.21418791716583946 
proporcion de  Masters con un salario mayor a 50k: 0.12485024816019168 
proporcion de  Preschool con un salario mayor a 50k: 8.5572